<a href="https://colab.research.google.com/github/stanfordstuckey/Classwork/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5: Statistical Analyses of Clinical Datasets

In this assignment you will gain experience analyzing preprocessed clinical datasets. You will practice using common time-saving tools in the `R` programming language that are ideally suited to these tasks. 

You will work with a dataset that we have prepared for you using a process similar to what you did in HW 3-4. The dataset describes patients from the [MIMIC III database](https://mimic.physionet.org/mimictables/patients/) who were put on mechanical ventilation and were stable for 12 hours. Some of these patients then experienced a sudden and sustained drop in oxygenation, while others did not. 

We have recorded a variety of features about each patient before the 12-hour mark (the index time), including counts of all prior diagnoses (aggregated with IC), all respiratory-related concepts in their notes, and indicators of events recorded in the patient charts. Indicator features are the number of times each event was recorded in the patient record, regardless of what the measured value was. For those chart events which have numeric values associated wtih them (e.g. lab tests) we found those in which a value was recorded for over 85% of the cohort and included the latest recorded value of those features. In addition, we have included demographic features (age and sex). For the small number of patients who did not have one or more of those features recorded, we used column-mean imputation to impute them. We also recorded whether or not each patient went on to experience a sudden and sustained drop in their oxygenation (the exposure). Finally, we recorded whether or not each patient eventually died during their hospitalization (the outcome). All of that data is contained in `patient_feature_matrix.csv`. Its companion file `feature_descriptions.csv` has descriptions of each of the features and their provenance. The final dataset you have access to is called `cohort.csv`, which contains the index time, exposure time (if any), in-hospital time of death (if any), and the time of censoring (when the patient was released from the hospital).

Please edit this document directly using either Jupyter Notebook or R markdown in R Studio and answer each of the questions below in-line. Jupyter and R markdown are useful tools for reproducible research that you will use over and over again in your later work. They are worth taking the short amount of time necessary to learn them. Turn in a single `.pdf` document showing all of your code and output for the entire assignment, with each question clearly demarcated. Submit your completed assignment through Canvas.

**Grading**: All answers will be graded on the correctness and quality of your code and analyses. Partial credit will be given based on a demonstration of conceptual understanding and how close you can come to solving the problem. At various points we will ask you to produce particular values: the correctness of these numbers will not be used for your grade - they are tools for us to get an idea about what your code is doing.

-----

## 0. Getting Ready

The first thing we need to do is load all of the important packages we will use for this assignment. Please load the packages  `caret`,  `ggplot2`, and `dplyr`. There are several other packages you will need or may want to use during the course of the assignment but if you need a package other than one of these three for a particular problem it will be noted in the problem statement.

-----

Next, load the CSV files `patient_feature_matrix.csv`, `cohort.csv` and `feature_descriptions.csv` as data frames.

-----

In [2]:
install.packages("caret")
library(caret)
library(ggplot2)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## 1. (5 pts) Preprocessing

### 1.1 (1 pts) Creating Feature Matrix and Outcome Vector

Split the patient matrix up into a numerical matrix of features and a character vector of the outcome (died or survived). For the feature matrix, exclude the subject ID and the outcome variable and use `data.matrix()`. 

----

### 1.2 (4 pts) Removing Uninformative Features

Before we do any modeling, let's cut down on our feature space by removing low-variance features that probably aren't useful enough to measure association with or use in a predictive model. `caret` has a function to do that, so let's use it instead of reinventing the wheel. 

Find the relevant function in the `caret` documentation and use it to create a new patient-feature matrix with only the useful features. From now on we will use the result of this step instead of the full feature matrix. Report how many of each different kind of feature are left after filtering out the near-zero variance features. As a sanity check, look at the kinds of features that are over-represented or under-represented in this set relative to the full set of features. Explain in a sentence if and why the result makes sense to you.

----

## 2. (55 pts) Associative Analyses

In this part of the assignment, you will use statistical tests to evaluate hypotheses about the relationship between  patient features and the binary outcome of whether a patient died during their ICU stay. You will also do a survival analysis using Kaplan-Meier curves and Cox regression to assess whether survival is significantly different between those who experienced a sudden and sustained drop in oxygenation, and those who did not.

-----

### 2.1 (12 pts) Hypothesis testing

#### 2.1.1 (10 pts) Statistical Tests of Differences Between Two Groups

For the features `alarms` (chart indicator), `activity` (chart indicator), `respiratory rate` (chart value), `arterial PaCO2` (chart value), `oxy_drop` (engineered feature) and `snomed ct concept` (note CUI), use a t-test, rank-sum test, Fisher exact test, or a $\chi^2$ (chi squared) test (whichever is most appropriate) to determine if each of these features is associated with mortality. Write your reasoning for determining which kind of test to use. If multiple tests are applicable to a comparison, use all of the applicable tests and compare the results. 

-----

#### 2.1.2 (2 pts) Hypothesis testing with Bonferroni correction

Use Bonferroni correction to determine the p-value cutoff if you were to evaluate association of all chart value features with death during ICU stay as an outcome. How many chart value features are significantly associated with death at this cutoff? How many chart value features are significantly associated with death (according to a t-test) at the standard cutoff of 0.05?

-----

### 2.2 (20 pts) Adjusted Analyses

In this part of the assignment you will build and compare several  regression models for the binary outcome of death during hospitalization.

-----

#### 2.2.1 (6 pts) Regression Models for Association

Use the `glm` package to build 3 models with the following independent variables. Use the kind of regression (set with the `family` parameter) that is appropriate for the data.

1. Age and oxy_drop
2. Age, gender and oxy_drop
3. Age, gender, oxy_drop and the chart value features that are signficantly associated with death after Bonferroni correction

-----

#### 2.2.2 (6 pts) Comparing regression models

What is the coefficient for `oxy_drop` in each model and what is its confidence interval? Why does the point estimate change as more features are added? Assuming you had a model of $Y$ regressed on $X_1$ and you added the variable $X_2$, under what conditions would the coefficient for $X_1$ not change? If both are positively correlated with the outcome and with each other, what would happen to the coefficient of $X_1$ after adding $X_2$? Why?

-----

#### 2.2.3 (4 pts) Legitimancy of Confidence Intervals

Assuming there are no systematic biases in the data and the only errors are from random sampling noise, do you think these confidence intervals are legitimate for all of these models, for none of them, or only for some of them? Explain your answer. If you said any of the confidence intervals are not legitimate, explain what you could change about the modeling procedure to make them so.

-----

#### 2.2.4 (4 pts) Goodness-of-fit testing

One way to compare models that use an increasing number of features is to test whether the residuals (the differences between the true outcome and the predicted outcome) are significantly different from each other. This is conceptually the same as assessing the likelihood of the data under each fitted model. In `R`, you can do that with the `anova` function by passing it a series of (generalized) linear models. Compare the 3 models you built in 2.2.1. Which model has the best fit (smallest deviance)? If we compared these three models using a held-out test set, would the same model necessarily have the lowest error? Why or why not?

-----

### 2.3 (23 pts) Survival Analysis

In this part of the assignment you will use your own code and `coxph` to fit survival (time-to-event) models. 

-----

#### 2.3.1 (3 pts) Creating Survival Data

Use the `cohort.csv` data to calculate the survival time (until death or censoring) for all patients. Use the `index_time`, `deathtime` and `censor_time` columns, as well as the function `mutate` to accomplish this. The time unit should be in days. Save these data in a new data frame called `patients_survival` that also keeps track of the `oxy_drop` value for each patient.

-----

#### 2.3.2 (13 pts) Kaplan-Meier Curves

Use your `patients_survival` data and `dplyr` and `ggplot2` to write your own code to generate Kaplan-Meier curves for patients who suffered a sudden and sustained drop in oxygenation and those who did not. Display both curves on the same plot in different colors. The functions `cumsum()` and `cumprod()` will likely come in handy. There are also some packages available that will calculate survival statistics and Kaplan-Meier plots for you, such as `survival` and `survminer`. Use functions in these packages to generate Kaplan-Meier curves for the survival data you created above. How do they compare to the curves generated with your code? What features do these packages provide that your code does not?

-----

#### 2.3.3 (7 pts) Cox Proportional Hazards Models

Use your `patients_survial` data combined with the patient feature matrix to run a univariate cox proportional hazards model of mortality regressed on a drop in oxygenation. Don't worry if you get a warning message about convergence. What is the value of the coefficient and its confidence interval? Also run a model adjusted for all of the non-zero variance features. What is the value of the coefficient for the drop in oxygenation and its confidence interval in that model? What is an explanation for the difference in the results?

-----

## 3. (40 pts) Causal Analyses

In our predictive analyses we saw that we could do a reasonable job of predicting mortality, but there are many cases where simply having a good prediction is not of much value. If clinicians are interested in saving the patient, they need to know what the causal factors are that lead to death so that they can intervene in the right place. Usually this question takes the form of comparing two exposures (drugs or procedures, for example), but we can also ask if the natural occurance of a condition causally leads to a bad outcome. Here we will investigate if having a sudden and sustained drop in oxygenation during ventilation (which we will refer to as the *exposure*) is causally related to death later in the hospitalization (the *outcome*).

### 3.1 (10 pts) Analyses without Matching

#### 3.1.1 (3 pts) Univariate Analysis

Run an appropriate statistical test to see if a drop in oxygenation is related to mortality. Treat both variables as binary.

What is the odds ratio? Does a drop in oxygenation appear to significantly decrease or increase the risk of death? Does this establish causality between a drop in oxygenation and mortality? Why or why not? If you think it does not, how else would you explain the result?

-----

#### 3.1.2 (5 pts) Multivariate Analysis

Let's see if we still see an effect after adjusting for other features. Run the appropriate kind of unregularized regression (using `glm`) of mortality on the same set of features as in the predictive modeling section (i.e. exclude near-zero variance features). You can use a formula like `died ~ .` to regress a variable named `died` in a data frame on the rest of the variables in that data frame.

Is the coefficient of the feature encoding a drop in oxygenation statistically significant in the model? What is the equivalent odds ratio? Does this establish causality between a drop in oxygenation and mortality? Why or why not? If you think it does not, how else would you explain the result?

-----

#### 3.1.3 (2 pts) Equivalent Experiment

Is there an experiment that you could run in theory that would let you determine the causal effect of a sudden oxygenation drop on mortality using only these inferential analyses? What might be some practical or ethical problems with your experiment?


### 3.2 (30 pts) Matching Analysis with Propensity Scores

#### 3.2.1 (5 pts) Propensity Modeling 

To try and more conclusively determine what the causal effect is we'll do a *propensity score* analysis. 

Estimate the exposure propensities of all the patients in the dataset using logistic regression on all non-zero-variance features, but of course exclude the feature that encodes the exposure (`oxy_drop`). Remember that now we are fitting a model to the *exposure* (`oxy_drop`) and not the *outcome*. 

-----

#### 3.2.2 (2 pts) Propensity Estimation 

Use your model to predict the propensity for each patient in the dataset. Use `qplot` to make a density plot (like a histogram) of the logits of the estimated propensities for all patients who truly did experience a sudden drop in oxygenation and an equivalant density plot for patients that did not. Diplay both curves in the same plot, coloring them differently. The `car` package has a `logit()` function you can use if you don't care to write your own.

What does the amount of overlap between these two distributions say about how good your propensity score model is predicting the exposure? Is it bad if your model has poor performance? (*hint: if the exposure happened totally at random, would that help you or hurt you in determining a causal effect?*)

-----

#### 3.2.3 (3 pts) Caliper Matching

The next step of a propensity score analysis is to match patients based on their propensities. Patients who have similar propensities are more "twin-like", so if one is exposed and the other isn't and then they have different outcomes, it's easier to say that the difference in exposure is what caused the different outcomes.

Use the `Match()` function from the `Matching` package to find a subset of the data where every exposed patient has a matched unexposed counterpart. Do the matching on the logit of the esimated propensity instead of on the propensity itself. Match without replacement and use a caliper (the maximum distance allowed between two matched patients) of 0.25. Your result should be a single vector containing the row numbers of the matched patients.

How many patients are in the matched set? How does this number compare to the orignal number of patients in the dataset? 

#### 3.2.4 (5 pts) Matched Outcomes Analysis

Repeat your statistical test from above (4.1.1) to see if the odds ratio between an drop in oxygenation is related to mortality, but this time use only data from the matched patients.

Is the estimated effect different than before? Is it significantly protective or harmful? How do you interpret this result?

-----

#### 3.2.5 (5 pts) Effect of Caliper

We could have had a stricter caliper and enforced that patients had to be more similar to be matched- what would have been the tradeoff in doing that in terms of the bias and variance of the estimate of the causal effect?

-----


#### 3.2.6 (5 pts) Including Post-Exposure Data in Propensity Models

In our dataset all of the information we have for each patient is from before the index time, which by definition is before the exposure time. What if we had included some data from after the exposure to fit the propensity score model? For instance, what if we included the total cost of the hospitalization as a feature that we used to estimate exposure propensities? Would our inference still be causally valid? Why or why not?

-----

#### 3.2.7 (5 pts) Lingering Biases

What are some sources of bias that our propensity score analysis cannot correct for? 

-----

## Feedback (0 points)
####  How much time did you spend on this assignment?

####  How much did you learn? Choose one (type your answer after the table):
   A | B | C | D | E |
   --|---|---|---|---|
   a great deal |  a lot  |  a moderate amount | a little | none at all|
   
#### Did you do any of the following: go to office hours, post on canvas, e-mail TAs? If so, which?

